In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- Configuration ---

DEFAULT_GRID_SIZE = 50
UPDATE_INTERVAL_MS = 100
MAX_HISTORY_SIZE = 10
DEFAULT_MAX_ITERATIONS = 500

# --- Initialization Modes ---
MODE_USER = "user"    # Start with a blank grid, waiting for user clicks and 'S' key.
MODE_RANDOM = "random" # Start immediately with a 20% random live cell population.

class GameOfLife(object):    
    '''
    The GameOfLife class implements the famous "Conway's Game of Life"
    on a finite grid with toroidal boundaries and loop detection.
    '''

    def __init__(self, size, max_iterations, initial_mode):
        """Initializes the game state based on the chosen mode."""
        self.size = size
        self.max_iterations = max_iterations
        
        self.life_grid = np.zeros((size, size), dtype=int)
        
        # Simulation state tracking
        self.iteration_count = 0
        self.grid_history = []
        self.ani_ref = None # Reference to the FuncAnimation object
        
        if initial_mode == MODE_RANDOM:
            self._initialize_random()
            self.is_setup_mode = False
            self.is_running = True
        else: # MODE_USER
            # Grid state starts as all dead (0) for user input
            self.is_setup_mode = True
            self.is_running = False
            
    def _initialize_random(self):
        """Helper to set the grid to a random state with 20% live cells."""
        self.life_grid = np.random.choice([0, 1], size=(self.size, self.size), p=[0.8, 0.2])

    def update_grid(self):
        """
        Applies the Game of Life rules using vectorized NumPy operations 
        for efficient computation.
        """
        current_grid = self.life_grid
        
        # 1. Count neighbors using rolling sums (handles toroidal boundaries)
        neighbors = (
            np.roll(current_grid, 1, 0) + np.roll(current_grid, -1, 0) +
            np.roll(current_grid, 1, 1) + np.roll(current_grid, -1, 1) +
            np.roll(np.roll(current_grid, 1, 0), 1, 1) + 
            np.roll(np.roll(current_grid, 1, 0), -1, 1) +
            np.roll(np.roll(current_grid, -1, 0), 1, 1) + 
            np.roll(np.roll(current_grid, -1, 0), -1, 1)
        )

        # 2. Apply rules using boolean logic
        
        # Survival/Death: Live cells (1) survive if neighbor count is 2 or 3.
        survival_mask = (current_grid == 1) & ((neighbors == 2) | (neighbors == 3))

        # Reproduction: Dead cells (0) become live if neighbor count is exactly 3.
        reproduction_mask = (current_grid == 0) & (neighbors == 3)

        # Combine the masks for the new grid
        next_grid = np.where(survival_mask | reproduction_mask, 1, 0)
        
        # Update the instance grid
        self.life_grid = next_grid
        
        return next_grid

    def check_termination(self, current_grid_flat):
        """Checks if the simulation should stop based on three conditions."""
        
        # 1. Check for maximum iteration limit
        if self.iteration_count >= self.max_iterations:
            return f"Max Iterations Reached ({self.max_iterations})"

        # 2. Check for Extinction (all dead)
        if not np.any(self.life_grid):
            return "Extinction (All cells died)"
        
        # 3. Check for Oscillation/Still Life (Loop Detection)
        if current_grid_flat in self.grid_history:
            period = self.grid_history.index(current_grid_flat) + 1
            return f"Loop Detected (Period {period})"
            
        return None # Simulation continues
    
    def handle_click(self, event, img):
        """Toggles the state of a cell when clicked during setup mode."""
        if not self.is_setup_mode or event.inaxes is None or event.xdata is None or event.ydata is None:
            return

        # Get grid coordinates (integers)
        col = int(round(event.xdata))
        row = int(round(event.ydata))

        # Check boundaries
        if 0 <= row < self.size and 0 <= col < self.size:
            # Toggle cell state (0 to 1, or 1 to 0)
            self.life_grid[row, col] = 1 - self.life_grid[row, col]
            
            # Update the plot immediately to show the change (live cells turn black)
            img.set_data(self.life_grid)
            img.figure.canvas.draw_idle()

    def handle_key(self, event, fig, img, ax):
        """Handles key presses, primarily to start the simulation."""
        if self.is_setup_mode and event.key.lower() == 's':
            print("--- Starting Simulation ('S' pressed) ---")
            self.is_setup_mode = False
            self.is_running = True
            
            # Disconnect setup event handlers
            for cid in self._cids:
                fig.canvas.mpl_disconnect(cid)

            # Update title to show simulation started and current generation
            ax.set_title(f"Conway's Game of Life (RUNNING) | Generation: {self.iteration_count}")

            # Start the animation
            self.start_animation(fig, img, ax)
            
            fig.canvas.draw_idle()
            
    def start_animation(self, fig, img, ax):
        """Starts the FuncAnimation loop."""
        ani = FuncAnimation(
            fig, 
            lambda frame: self.tick(img, ax), # Pass ax for title updates
            frames=None, 
            interval=UPDATE_INTERVAL_MS, 
            blit=False 
        )
        self.ani_ref = ani
        
    def tick(self, img, ax):
        """
        Advances the simulation by one generation and checks for termination.
        This method is designed to be called by FuncAnimation.
        """
        if not self.is_running:
            if self.ani_ref:
                self.ani_ref.event_source.stop()
            # Only return artists (img)
            return [img] 

        # --- PRE-TICK CHECK (Termination based on previous state) ---
        current_grid_flat = self.life_grid.flatten().tobytes()
        
        # Check termination conditions based on the CURRENT state
        termination_message = self.check_termination(current_grid_flat)
        
        if termination_message:
            self.is_running = False
            
            # Update title one last time with the termination message
            final_title = f"TERMINATED: {termination_message} | Final Generation: {self.iteration_count}"
            ax.set_title(final_title, color='red')
            print(f"--- TERMINATED after {self.iteration_count} generations: {termination_message} ---")
            
            # Stop the animation timer
            if self.ani_ref:
                self.ani_ref.event_source.stop()

            # Only return artists (img)
            return [img] 

        # --- ADVANCE GENERATION ---
        
        # Record the current state as a 'previous' state for the next check
        self.grid_history.append(current_grid_flat)
        if len(self.grid_history) > MAX_HISTORY_SIZE:
            self.grid_history.pop(0)

        self.iteration_count += 1
        
        # Calculate next grid state
        self.update_grid()
        
        # Update plot artists
        img.set_data(self.life_grid)
        # Update the figure title with the current generation count
        ax.set_title(f"Conway's Game of Life (RUNNING) | Generation: {self.iteration_count}", color='black')
        
        # Only return artists (img)
        return [img]



def get_integer_input(description, min_value, default):
    """
    Helper function to get and validate positive integer input, 
    displaying the default value clearly.
    """
    while True:
        try:
            
            value = input(f"{description} (Default: {default}): ")
            if not value:
                return default
            
            int_value = int(value)
            if int_value < min_value:
                print(f"Value must be greater than or equal to {min_value}.")
            else:
                return int_value
        except ValueError:
            print("Invalid input. Please enter a whole number.")
        except EOFError:
            return default # Return default if input stream closes

def get_mode_input():
    """Helper function to get and validate simulation mode selection input."""
    print(f"Choose Simulation Mode: ('U' for User Setup, 'R' for Random Start)")
    while True:
        try:
            choice = input("Enter U or R: ").strip().upper()
            if choice == 'U':
                return MODE_USER
            elif choice == 'R':
                return MODE_RANDOM
            else:
                print("Invalid input. Please enter 'U' or 'R'.")
        except EOFError:
            print("\nInput stream closed. Defaulting to User Setup mode.")
            return MODE_USER
        except Exception as e:
            print(f"An error occurred during input: {e}. Defaulting to User Setup mode.")
            return MODE_USER



def run_simulation():
    
    # Get runtime inputs from the user
    print("--- Conway's Game of Life Initialization ---")
    
    # 1. Get Grid Size
    user_grid_size = get_integer_input("Enter Grid Size", min_value=1, default=DEFAULT_GRID_SIZE)
    
    # 2. Get Max Iterations
    user_max_iterations = get_integer_input("Enter Max Iterations", min_value=1, default=DEFAULT_MAX_ITERATIONS)
    
    # 3. Get Initialization Mode
    INITIAL_MODE = get_mode_input()
    
    # 4. Setup Game and Matplotlib
    game = GameOfLife(user_grid_size, user_max_iterations, INITIAL_MODE)
    
    # Use user_grid_size for figure sizing
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Initialize the image artist with the starting grid
    img = ax.imshow(game.life_grid, interpolation='nearest', cmap='binary', vmin=0, vmax=1) 
    
    # --- Grid Lines for visual selection feedback ---
    ax.set_xticks(np.arange(-0.5, user_grid_size, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, user_grid_size, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.tick_params(which='minor', size=0)
    ax.set_xlim(-0.5, user_grid_size - 0.5)
    ax.set_ylim(user_grid_size - 0.5, -0.5) # Invert Y-axis
    # --- END Grid Lines ---

    if game.is_setup_mode:
        title_text = "Game of Life SETUP: Click cell to toggle, Press 'S' to start"
    else:
        # For random mode, start with the current status in the title
        title_text = f"Conway's Game of Life (RUNNING) | Generation: {game.iteration_count}"
    
    # Set the initial title
    ax.set_title(title_text) 
    
    # 5. Connect Event Handlers or Start Animation
    if game.is_setup_mode:
        # User Mode: Connect setup event handlers
        game._cids = []
        click_cid = fig.canvas.mpl_connect('button_press_event', lambda event: game.handle_click(event, img))
        key_cid = fig.canvas.mpl_connect('key_press_event', lambda event: game.handle_key(event, fig, img, ax))
        game._cids.append(click_cid)
        game._cids.append(key_cid)
    else:
        # Random Mode: Start immediately
        game.start_animation(fig, img, ax)

    # Show the window
    plt.show()

if __name__ == '__main__':
    run_simulation()

--- Conway's Game of Life Initialization ---
Enter Grid Size (Default: 50): 20
Enter Max Iterations (Default: 500): 50
Choose Simulation Mode: ('U' for User Setup, 'R' for Random Start)
Enter U or R: r
--- TERMINATED after 26 generations: Loop Detected (Period 9) ---
